# 数据集下载

数据集从比赛链接处下载：


<img src="https://ai-studio-static-online.cdn.bcebos.com/6303c3a35cc548fc9b01037fb41ef491c10c85ea62f74427ab9fb52b99b58ee1" width = "800" height = "800" alt="" align=center />



在/home/aistudio/data 目录下已经包含了下载好的训练数据和测试数据。

# 解压数据集

使用tree命令可以查看到训练集位于/home/aistudio/data 目录下，可以进入到这个目录下解压训练数据

In [ ]:
# 检查数据集所在路径
!tree -L 3 /home/aistudio/data

In [18]:
!cd /home/aistudio/data/data87746 && unzip  训练数据集.zip
!cd /home/aistudio/data/data87750 && unzip  A榜测试数据集.zip

In [ ]:
!cd /home/aistudio/work/Ultra_light_OCR_No.23 && unzip  upload.zip

使用命令查看训练数据文件夹下数据量是否是10万张

In [ ]:
!cd ~/data/data87746/训练数据集/TrainImages  &&  ls -l | grep "^-" | wc -l

# 下载代码


In [ ]:
# 进入到gitclone 的目录下
%cd /home/aistudio/work/

# 从从GitHub链接下载代码
!git clone https://github.com/TangJiamin/Ultra_light_OCR_No.23.git
# 检查源代码文件结构
# !cd work; mkdir model
!tree /home/aistudio/work/ -L 2

# 代码内容说明


## 数据拼接

对较短的文本进行拼接操作，拼接后的文本默认保存在'save_path'中，拼接后的标签默认保存在'./concat_train_label.txt'中

In [19]:
# 进入gitclone 的data_process目录下
%cd /home/aistudio/work/Ultra_light_OCR_No.23/data_process/
# 运行concat.py文件，修改训练数据和标签路径
!python3.7 concat.py

/home/aistudio/work/Ultra_light_OCR_No.23/data_process
100%|████████████████████████████████████| 57947/57947 [01:40<00:00, 576.61it/s]


使用拼接后得到的图片替换原有训练数据集中相应图片得到新的训练数据集

In [22]:
import shutil
import os
target_path = '/home/aistudio/data/data87746/训练数据集/TrainImages'
ori_path = '/home/aistudio/data/data87746/训练数据集/concat_images'
for file in os.listdir(ori_path):
    shutil.copy(os.path.join(ori_path, file), os.path.join(target_path, file))

## Dockerfile 创建容器


In [10]:
# 进入到gitclone 的目录下
%cd /home/aistudio/work/Ultra_light_OCR_No.23/
# build 镜像
!docker build -t paddleocr .
# 创建容器时挂载数据
!docker run -it -v /home/aistuodio/data:/data --gpus all paddleocr:latest /bin/bash

## 基于PaddleOCR的识别模型训练

In [12]:
# 进入到容器的PaddleOCR目录下
%cd /home/aistudio/work/Ultra_light_OCR_No.23/PaddleOCR/

/home/aistudio/work/Ultra_light_OCR_No.23/PaddleOCR


配置环境

In [16]:
!pip install -r requirements.txt -i https://mirror.baidu.com/pypi/simple

设置配置文件，完成识别算法训练 PaddleOCR 通过yml配置文件的方式选择不同的算法和训练参数等，这里我们使用configs/rec/ch_ppocr_v2.0/rec_chinese_lite_train_v2.0.yml配置文件完成识别算法训练；

从该配置文件中，我们可以得到如下信息：

Architecture: model_type: rec algorithm: CRNN Transform: Backbone: name: MobileNetV3 scale: 0.5 model_name: large large_stride: [1, 2, 2, 2] Neck: name: SequenceEncoder encoder_type: rnn hidden_size: 48 Head: name: CTCHead fc_decay: 0.00001

Loss: name: CTCLoss 表示我们使用的是CRNN算法，backbone是MobileNetV3，损失函数是CTCLoss。我们也可以在配置文件中设置训练数据和测试数据的路径，

vim configs/rec/ch_ppocr_v2.0/rec_chinese_lite_train_v2.0.yml

对于其中的Train.data_dir, Train.label_file_list, Eval.data_dir, Eval.label_file_list字段需要用户手动配置相关路径，

本项目中路径如下所示：

Train: dataset: name: SimpleDataSet data_dir: /home/tangjiamin/PaddleOCR/train_data/TrainImages label_file_list: ["/home/tangjiamin/PaddleOCR/train_data/concat_train_label.txt"] ... ...

Eval: dataset: name: SimpleDataSet data_dir: /home/tangjiamin/PaddleOCR/test_data/TestAImages label_file_list: ["/home/tangjiamin/PaddleOCR/test_data/ppocr_server_v2.0_rec_infer.txt"] 另外，本实验仅做示例参考，测试集reader部分使用训练集数据和标签，所以，也关闭训练中评估的选项，即将Global.cal_metric_during_train 设置为False。

Global: use_gpu: true ..... cal_metric_during_train: False 开发者们在做实验时，可以把训练数据拆分为训练集和测试集。

由于aistuido的共享内存有限，可能无法很好支持多线程的任务，所以需要将以下字段改成0

Train.loader.num_workers: 0 Eval.loader.num_workers: 0 关于yml的更多细节，可以参考PaddleOCR

## 训练脚本

本代码会训练一个模型，并且保存到Global.save_model_dir中，本实验中，训练日志默认保存到文件`output/rec_chinese_lite_v2.0/train.log`   

### 训练日志样例
```
[2021/06/08 10:27:22] root INFO: epoch: [496/1000], iter: 10, lr: 0.000007, loss: 3.031638, acc: 0.653333, norm_edit_dis: 0.927252, reader_cost: 1.45727 s, batch_cost: 2.27184 s, samples: 1650, ips: 72.62830
[2021/06/08 10:27:31] root INFO: epoch: [496/1000], iter: 20, lr: 0.000007, loss: 3.199826, acc: 0.640000, norm_edit_dis: 0.925748, reader_cost: 0.00378 s, batch_cost: 0.51155 s, samples: 1500, ips: 293.22549
[2021/06/08 10:27:39] root INFO: epoch: [496/1000], iter: 30, lr: 0.000007, loss: 3.371130, acc: 0.630000, norm_edit_dis: 0.923354, reader_cost: 0.00290 s, batch_cost: 0.52430 s, samples: 1500, ips: 286.09623
[2021/06/08 10:27:49] root INFO: epoch: [496/1000], iter: 40, lr: 0.000007, loss: 3.372990, acc: 0.636667, norm_edit_dis: 0.923633, reader_cost: 0.00280 s, batch_cost: 0.57715 s, samples: 1500, ips: 259.89564
[2021/06/08 10:27:58] root INFO: epoch: [496/1000], iter: 50, lr: 0.000007, loss: 3.407562, acc: 0.640000, norm_edit_dis: 0.924480, reader_cost: 0.00350 s, batch_cost: 0.53732 s, samples: 1500, ips: 279.16580
```

启动训练前要修改使用的config中字典表路径、训练和测试数据及标签路径，启动训练代码如下：

In [17]:
!python3.7 tools/train.py -c configs/rec/ch_ppocr_v2.0/rec_chinese_lite_train_v2.0.yml

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Config use_gpu cannot be set as true while you are using paddlepaddle cpu version ! 
Please try: 
	1. Install paddlepaddle-gpu to run model on GPU 
	2. Set use_gpu as false in config file to run model on CPU


## 基于预测引擎的预测
本代码会预测训练好的模型，并将结果保存成txt格式，可以直接送到比赛提交入口测评，文件默认保存在`/infer_pre_res.txt` 

#### 样例
`python3.7 tools/infer/predict_rec.py --image_dir="./test_data/TestAImages" --rec_model_dir="./output/bd_competition/best/inference" --rec_image_shape="3, 48, 512" --rec_char_type="ch" --rec_char_dict_path="./train_data/charset.txt""`

#### 你将会看到类似如下的测试日志
```
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009988.jpg:('不天而心', 0.9524222)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009989.jpg:('高于国家标准', 0.99632424)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009990.jpg:('无', 0.92225355)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009991.jpg:('科目', 0.92987084)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009992.jpg:('理', 0.9996427)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009993.jpg:('东航MU54171319（中）', 0.9353286)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009994.jpg:('极润护肤', 0.99700654)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009995.jpg:('金凤区分局', 0.99352777)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009996.jpg:('零售价：', 0.99186826)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009997.jpg:('5', 0.9993771)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009998.jpg:('廿四', 0.99556893)
[2021/07/08 15:05:06] root INFO: Predicts of ./test_data/TestBImages/TestB_009999.jpg:('3.5折C', 0.9477636)
[2021/07/08 15:05:06] root INFO: Total predict time for 10000 images, cost: 78.094
```

In [ ]:
!du -sh ./inference/

inference模型保存在'./inference'文件夹，可使用du命令查看导出的inference模型大小。

启动预测代码如下：

In [ ]:
!python3.7 tools/infer/predict_rec.py --image_dir="./test_data/TestAImages" --rec_model_dir="./output/bd_competition/best/inference" --rec_image_shape="3, 48, 512" --rec_char_type="ch" --rec_char_dict_path="./charset.txt"

# 模型构建思路及调优过程

## 数据增强策略

对训练图片依次执行tia_distort、tia_stretch、tia_perspective、剪切（训练一段时间后取消）、模糊、变色、抖动、噪声和反色操作，每个操作执行的概率为0.4

## 调参优化策略

初始学习率设为0.003，数据增强策略中包含剪切操作，在迭代一段时间后发现loss开始震荡，将学习率降至0.001并取消剪切操作